# NARM+




In [2]:
#Importing functions.
import pandas
import csv
from collections import defaultdict
from collections import namedtuple
import itertools
import math
import numpy as np
import pandas
import random
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [14]:
#Don't Run this cell if you don't want to read the entire database.
readDF = pandas.read_csv('./data/UserBehavior.csv', names=['USERID', 'PRODUCTID', 'CATEGORYID', 'ACTION', 'TIMESTAMP'])
group = readDF.groupby("USERID")
sorted(readDF.USERID.unique())[0:100]


[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 53,
 54,
 55,
 57,
 58,
 59,
 61,
 62,
 63,
 64,
 65,
 66,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108]

In [15]:
#Don't rn This cell if you haven't read the entire database.
smallTaoBao = readDF.loc[(readDF['USERID'] >= 1) & (readDF['USERID'] <= 108)]
smallTaoBao.to_pickle("./data/smallTaoBao.pkl")
unpickled_smallTaoBao = pandas.read_pickle("./data/smallTaoBao.pkl")
unpickled_smallTaoBao

In [4]:
# Create sessions
unpickled_smallTaoBao = pandas.read_pickle("./data/smallTaoBao.pkl")
unpickled_smallTaoBao
unpickled_smallTaoBao['SESSION'] = pandas.to_datetime(unpickled_smallTaoBao['TIMESTAMP'],unit='s').dt.date
unpickled_smallTaoBao.head()

,USERID,PRODUCTID,CATEGORYID,ACTION,TIMESTAMP,SESSION
0,1,2268318,2520377,pv,1511544070,2017-11-24
1,1,2333346,2520771,pv,1511561733,2017-11-24
2,1,2576651,149192,pv,1511572885,2017-11-25
3,1,3830808,4181361,pv,1511593493,2017-11-25
4,1,4365585,2520377,pv,1511596146,2017-11-25


In [5]:
print("Average number of sessions per user")
unpickled_smallTaoBao.groupby('USERID')['SESSION'].nunique().mean()

Average number of sessions per user


7.2

In [7]:
print("Average number of clicks per session")
unpickled_smallTaoBao.groupby(['USERID', 'SESSION'])['ACTION'].count().mean()

Average number of clicks per session


14.126388888888888

In [17]:
user1 = unpickled_smallTaoBao.loc[unpickled_smallTaoBao['USERID'] == 1]
user1

,USERID,PRODUCTID,CATEGORYID,ACTION,TIMESTAMP,SESSION
0,1,2268318,2520377,pv,1511544070,2017-11-24
1,1,2333346,2520771,pv,1511561733,2017-11-24
2,1,2576651,149192,pv,1511572885,2017-11-25
3,1,3830808,4181361,pv,1511593493,2017-11-25
4,1,4365585,2520377,pv,1511596146,2017-11-25
5,1,4606018,2735466,pv,1511616481,2017-11-25
6,1,230380,411153,pv,1511644942,2017-11-25
7,1,3827899,2920476,pv,1511713473,2017-11-26
8,1,3745169,2891509,pv,1511725471,2017-11-26
9,1,1531036,2920476,pv,1511733732,2017-11-26


In [41]:
IDs = list(unpickled_smallTaoBao.USERID.unique())
sessions = sorted(list(unpickled_smallTaoBao.SESSION.unique()))
userBase = defaultdict(list)

for ID in IDs:
    userData = unpickled_smallTaoBao.loc[unpickled_smallTaoBao['USERID'] == ID]
    
    #Verschil tussen historical data and current session.
    
    for session in sessions:
        userSession = userData.loc[userData['SESSION'] == session]
        userData_PRODUCTLIST = userSession['PRODUCTID'].values.tolist()
        
        if len(userData_PRODUCTLIST) > 0:
            userBase[ID].append(userData_PRODUCTLIST)
    
    

print(userBase[1])

[[2268318, 2333346], [2576651, 3830808, 4365585, 4606018, 230380], [3827899, 3745169, 1531036], [2266567, 2951368, 3108797, 1338525, 2286574], [5002615, 2734026, 5002615, 3239041, 4615417, 4152983, 266784, 46259, 266784, 4092065, 1305059], [2791761, 3239041, 46259, 4973305, 2087357, 3157558], [2087357, 4170517, 1340922, 3911125, 4170517, 3682069, 4954999, 79715, 4666650], [1323189, 4198227, 4954999], [2041056, 3219016, 2104483, 2028434, 3219016, 2278603, 929177], [4954999, 818610, 271696, 568695]]


In [46]:
# DON'T RUN THIS YET, NEEDS REWORKING

# Create dataset of examples

# A simple way to define a class is using namedtuple.
Example = namedtuple("Example", ["userID", "history", "inputs", "target"])

def createExamples(userBase, IDs):
    # convert raw data into examples and create dataset
    
    #TODO: create train and testdata
    #Example = userID, history, inputs, targets?
    
    trainData = []
    testData = []
    
    for ID in IDs:
        userData = userBase[ID]
        historyTrain = userData[:-2]
        inputsTrain = userData[len(userData) -2]
        trainData.append(Example(userID = ID, history = historyTrain, inputs=inputsTrain[:-1],target=inputsTrain[1:]))
        
        historyTest = userData[:-1]
        inputsTest = userData[len(userData) - 1]
        testData.append(Example(userID = ID, history = historyTest, inputs=inputsTest[:-1], target=inputsTest[1:]))
        
    return trainData, testData

trainData, testData = createExamples(userBase, IDs)
print(trainData[0])
print(testData[0])


Example(userID=1, history=[[2268318, 2333346], [2576651, 3830808, 4365585, 4606018, 230380], [3827899, 3745169, 1531036], [2266567, 2951368, 3108797, 1338525, 2286574], [5002615, 2734026, 5002615, 3239041, 4615417, 4152983, 266784, 46259, 266784, 4092065, 1305059], [2791761, 3239041, 46259, 4973305, 2087357, 3157558], [2087357, 4170517, 1340922, 3911125, 4170517, 3682069, 4954999, 79715, 4666650], [1323189, 4198227, 4954999]], inputs=[2041056, 3219016, 2104483, 2028434, 3219016, 2278603], target=[3219016, 2104483, 2028434, 3219016, 2278603, 929177])
Example(userID=1, history=[[2268318, 2333346], [2576651, 3830808, 4365585, 4606018, 230380], [3827899, 3745169, 1531036], [2266567, 2951368, 3108797, 1338525, 2286574], [5002615, 2734026, 5002615, 3239041, 4615417, 4152983, 266784, 46259, 266784, 4092065, 1305059], [2791761, 3239041, 46259, 4973305, 2087357, 3157558], [2087357, 4170517, 1340922, 3911125, 4170517, 3682069, 4954999, 79715, 4666650], [1323189, 4198227, 4954999], [2041056, 3219

In [ ]:
# HELPER FUNCTIONS

# function to yield one example at a time
def get_examples(data, shuffle=True, **kwargs):
    """Shuffle data set and return 1 example at a time (until nothing left)"""
    if shuffle:
#         print("Shuffling training data")
        random.shuffle(data)  # shuffle training data each epoch
    for example in data:
        yield example
    
# function to prepare an example for usage by the model
def prepare_example(example):
    """
    Turn an example into tensors of inputs and target.
    """
    v = torch.FloatTensor([example.userID])
    v = v.to(device)
    
    w = torch.FloatTensor([example.history])
    w = w.to(device)
    
    x = torch.FloatTensor([example.inputs])
    x = x.to(device)

    y = torch.FloatTensor([example.target])
    y = y.to(device)

    return v, w, x, y

# function to yield a (mini-)batch
def get_minibatch(data, batch_size=25, shuffle=True):
    """Return minibatches, optional shuffling""" 
    if shuffle:
#         print("Shuffling training data")
        random.shuffle(data)  # shuffle training data each epoch

    batch = []

    # yield minibatches
    for example in data:
        batch.append(example)

        if len(batch) == batch_size:
            yield batch
            batch = []    
        # in case there is something left
        if len(batch) > 0:
            yield batch

# function to make the (mini-)batch ready for usage by the model
def prepare_minibatch(mb):
    """
    Minibatch is a list of examples.
    This function converts returns
    torch tensors to be used as input/targets.
    """
    batch_size = len(mb)
    x = [ex.inputs for ex in mb]
    x = torch.FloatTensor(x)
    x = x.to(device)

    y = [ex.target for ex in mb]
    y = torch.FloatTensor(y)
    y = y.to(device)

    return x, y

# simple evaluation function
def simple_evaluate(model, data, prep_fn=prepare_example, **kwargs):
    """Explained Variance Score of a model on given data set."""
    model.eval()  # disable dropout (explained later)
    targets = []
    predictions = []

    for example in data:

        # convert the example input and label to PyTorch tensors
        targets.append(example.target)
        x, target = prepare_example(example)

        # forward pass
        # get the output from the neural network for input x
        with torch.no_grad():
            output = model(x)
        # output shape: (batch, output_size)
        prediction = output[0].tolist()
        predictions.append(prediction)
            
    score = explained_variance_score(targets, predictions, multioutput='variance_weighted')

    return score, None

In [ ]:
# Custom NN

class NarmPlus(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_size, output_dim, num_layers, activation_fn=nn.RReLU(),
                num_items):
        super(NarmPlus, self).__init__()
        self.input_dim = input_dim
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.hidden_state_dim = 0 # calculation needed
        self.ItemEmbedding = nn.Embedding(num_items, embedding_dim)
        self.ActivationFn = activation_fn
        self.UserProfile = nn.Linear(embedding_dim, self.hidden_state_dim)
        self.Local = nn.GRU()
        self.Global = nn.GRU()
        self.Softmax = nn.Softmax()
        self.Decoder = nn.Bilinear(embedding_dim, , output_dim
        
    def forward(self,x):
        history, current = x
        embeds_h = self.ItemEmbedding(history)
        profile = self.UserProfile(embeds_h)
        
        
        c = torch.cat(c_global, c_local, dim=2)
        out = self.Decoder()
        output = self.Softmax(out)
        return output
        